# Recipes to perform Nvidia GPU INT-8 quantization on most transformers model

Quantization is one of the most effective and generic approach to make model inference faster.
Basically, it replaces high precision float numbers in model tensors encoded in 32 or 16 bits by lower precision ones encoded in 8 bits or less:

* it takes less memory
* computation is easier / faster

It can be applied to any model in theory, and, if done well, it should not decrease model accuracy.

The purpose of this tutorial is to show 2 processes to perform quantization on most `transformer` architecture.

**TL;DR, inference is 5 times faster on a `Roberta-base` model** with a batch of size 32 / seq len 384, benchmark (bold, quantization):

| Framework                  | Precision | Latency (ms) | Accuracy | Speedup |
| -------------------------- | --------- | ------------ | -------- | ------- |
| Pytorch                    | FP32      | 76.31        | ---      | X 1     |
| Pytorch                    | FP16      | 56.04        | ---      | X 1.4   |
| TensorRT                   | FP16      | 30.17        | ---      | X 2.5   |
| TensorRT (transplantation) | **INT-8** | 15.83        | ---      | **X 5** |
| TensorRT (custom QDQ code) | **INT-8** | 14.94        | ---      | **X 5** |

> same kind of acceleration is observed on all seq len / batch sizes


## A (very) short intro to INT-8 quantization

Basic idea behind model quantization is to replace tensors made of float numbers (usually encoded on 32 bits) by lower precision representation (encoded on 8 bits for Nvidia GPUs), in general integers.
Therefore computation is faster and model memory footprint is lower. Making tensor storage smaller makes memory transfer faster... and is also a computation acceleration factor.
This technic is very interesting for its trade-off: you reduce inference time significantly, and in most scenarios it cost close to nothing in accuracy.

Replacing float numbers by integers is done through a mapping.
This step is called `calibration`, and its purpose is to compute for each tensor or each channel of a tensor (one of its dimensions) a range of all possible values and then define a scale and a distribution center to map float numbers to 8 bits integers.
The process is well described in this [Nvidia presentation](https://on-demand.gputechconf.com/gtc/2017/presentation/s7310-8-bit-inference-with-tensorrt.pdf).

There are several ways to perform quantization, depending of how and when the `calibration` is performed:

* dynamically: the mapping is done during the inference, there are some overhead but it's easy to put in place and usually the accuracy is preserved,
* statically, after training (`post training quantization` or `PTQ`): this way is efficient, but it may have a significant accuracy cost,
* statically, before training (`quantization aware training` or `QAT`): this way is efficient and has a low accuracy cost as the weights will take care of the result

In this guide we will focus on the third option: QAT.

During a quantization aware training:

* in the inside, Pytorch will work with high precision float numbers,
* on the outside, Pytorch will simulate that a quantization has already been applied and output results accordingly (for loss computation for instance)
* it will also refine the quantization mapping (scale, range, distribution center, etc.)

You can check this [high quality blog post](https://leimao.github.io/article/Neural-Networks-Quantization/) for more information.

## Why a dedicated tutorial?

CPU quantization is supported out of the box by `Pytorch` or `ONNX Runtime`.
**GPU quantization on the other side requires specific tools and process to be applied**.

In the specific case of `transformer` models, right now (december 2021), the only way shown by Nvidia is to build manually the graph of our models in `TensorRT`. This is a low level approach, based on GPU capacity knowledge (which operators are supported, etc.). It's certainly out of reach of most NLP practitioners and is very time consuming to update/adapt to new architectures.

Hopefully, Nvidia recently added to Hugging Face `transformer` library a new model called `QDQBert`.
Basically, it's a vanilla `Bert` architecture which supports INT-8 quantization.
It doesn't support any other architecture out of the box, like `Albert`, `Roberta`, or `Electra`.
Nvidia also provide a demo dedicated to the SQuaD task.

To be both simple and cover most use cases, in this tutorial we will see:

* how to perform GPU quantization on **any** transformer model (not just Bert) using a simple trick, a `transplatation`
* how to perform GPU quantization on `QDQRoberta`, a custom model similar to `QDQBert` and supported by `transformer-deploy` library
* how to apply quantization to a common task like classification (which is easier to understand than question answering)
* measure performance gain (latency)


## Project setup

### Dependencies installation

We install `master` branch of `transfomers` library to use a new model: **QDQBert** and `transformer-deploy` to leverage `TensorRT` models (TensorRT API is not something simple to master, it's highly advised to use a wrapper).

In [1]:
#! pip install git+https://github.com/huggingface/transformers
#! pip install git+https://github.com/ELS-RD/transformer-deploy
#! pip install sklearn datasets
#! pip install pytorch-quantization --extra-index-url https://pypi.ngc.nvidia.com
# or install pytorch-quantization from https://github.com/NVIDIA/TensorRT/tree/master/tools/pytorch-quantization

Check the GPU is enabled and usable.

In [2]:
! nvidia-smi

Tue Dec  7 21:16:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:03:00.0  On |                  N/A |
| 35%   42C    P8    40W / 350W |    221MiB / 24267MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import numpy as np
from tqdm.notebook import tqdm

from typing import OrderedDict as OD, List, Dict
import torch
from torch import Tensor
from transformers import (
    AutoModelForSequenceClassification,
    PreTrainedModel,
    QDQBertForSequenceClassification,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
    IntervalStrategy,
)
from transformer_deploy.QDQModels.QDQRoberta import QDQRobertaForSequenceClassification
import pytorch_quantization.nn as quant_nn
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import calib
import logging
import transformers
import datasets
from datasets import DatasetDict
from transformer_deploy.backends.trt_utils import build_engine, get_binding_idxs, infer_tensorrt, load_engine
from transformer_deploy.backends.ort_utils import convert_to_onnx
from collections import OrderedDict
from transformer_deploy.benchmarks.utils import track_infer_time, print_timings

In [4]:
from pycuda._driver import Stream
import tensorrt as trt
from tensorrt.tensorrt import IExecutionContext, Logger, Runtime
import pycuda.autoinit

Set logging to `error` to make the `notebook` more readable on Github.

In [5]:
log_level = logging.ERROR
logging.getLogger().setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
trt_logger: Logger = trt.Logger(trt.Logger.ERROR)
transformers.logging.set_verbosity_error()

### Download data

This part is inspired from an [official Notebooks from Hugging Face](https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb).

In [6]:
task = "mnli"
num_labels = 3
model_checkpoint = "roberta-base"
batch_size = 32
max_seq_len = 256
validation_key = "validation_matched"
timings: Dict[str, List[float]] = dict()

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.

In [7]:
from datasets import load_dataset, load_metric

dataset = load_dataset("glue", task)
metric = load_metric("glue", task)
dataset

  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

### Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [8]:
from transformers import AutoTokenizer, PreTrainedTokenizer

tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [9]:
def preprocess_function(examples):
    return tokenizer(
        examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=max_seq_len
    )

In [10]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Some functions required for training and exporting the model:

In [48]:
from typing import Union


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


def calibrate(model: PreTrainedModel, encoded_dataset: DatasetDict, nb_sample: int = 128) -> PreTrainedModel:
    # Find the TensorQuantizer and enable calibration
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.disable_quant()
                module.enable_calib()
            else:
                module.disable()

    with torch.no_grad():
        for start_index in tqdm(range(0, nb_sample, batch_size)):
            end_index = start_index + batch_size
            data = encoded_dataset["train"][start_index:end_index]
            input_torch = {
                k: torch.tensor(v, dtype=torch.long, device="cpu")
                for k, v in data.items()
                if k in ["input_ids", "attention_mask", "token_type_ids"]
            }
            model(**input_torch)

    # Finalize calibration
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                if isinstance(module._calibrator, calib.MaxCalibrator):
                    module.load_calib_amax()
                else:
                    module.load_calib_amax("percentile", percentile=99.99)
                module.enable_quant()
                module.disable_calib()
            else:
                module.enable()

    model.cuda()
    return model


def convert_tensor(data: OD[str, List[List[int]]], output: str) -> OD[str, Union[np.ndarray, torch.Tensor]]:
    input: OD[str, Union[np.ndarray, torch.Tensor]] = OrderedDict()
    for k in ["input_ids", "attention_mask", "token_type_ids"]:
        if k in data:
            v = data[k]
            if output == "torch":
                value = torch.tensor(v, dtype=torch.long, device="cuda")
            elif output == "np":
                value = np.asarray(v, dtype=np.int32)
            else:
                raise Exception(f"unknown output type: {output}")
            input[k] = value
    return input

In [12]:
runtime: Runtime = trt.Runtime(trt_logger)
profile_index = 0

## Fine-tuning model

Now that our data are ready, we can download the pretrained model and fine-tune it.

We will also prepare some export function right now

Default parameters to be used for the training:

In [13]:
nb_step = 1000
strategy = IntervalStrategy.STEPS
args = TrainingArguments(
    f"{model_checkpoint}-{task}",
    evaluation_strategy=strategy,
    eval_steps=nb_step,
    logging_steps=nb_step,
    save_steps=nb_step,
    save_strategy=strategy,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size * 2,
    num_train_epochs=1,
    fp16=True,
    group_by_length=True,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to=[],
)

## Method 1: `Transplantation` of weights from a source model to an optimized architecture

Transplantation idea is to export weights from one model and use them in another one.
In our case, the source are `Roberta` weights and the target is `Bert` archtecture which is highly optimized on `TensorRT` for GPU quantization.

Indeed, not all models are quantization compliant. The optimization engine (`TensorRT`) search for some patterns and will fail to opimize the model if it doesn't find them. It requires the Pytorch code to be written in a certain way and use certain operations. For that reason, it's a good idea to reuse an architecture highly optimized.

We will leverage the fact that since `Bert` have been released, very few improvements have been brought to the transformer architecture (at least for encoder only models).
Better models appeared, and most of the work has been done to improve the pretraining step (aka the weights).
So the idea will be to take the weights from those new models and put them inside `Bert` architecture.

The process described below should work for most users.

**steps**:

* load `Bert` model
* retrieve layer/weight names
* load target model (here `Roberta`)
* replace weight/layer names with those from `Roberta`
* override the architecture name in model configuration

If there is no 1 to 1 correspondance (it happens), try to keep at least embeddings and self attention. Of course, it's possible that if a model is very different, the transplant may cost some accuracy. In our experience, if your trainset is big enough it should not happen.


In [14]:
model_bert: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=num_labels
)
bert_keys = list(model_bert.state_dict().keys())
del model_bert

model_roberta: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels
)
model_roberta.save_pretrained("roberta-in-bert")
del model_roberta
model_weights: OD[str, Tensor] = torch.load("roberta-in-bert/pytorch_model.bin")

# Roberta -> Bert, there is 1 to 1 correspondance, for other models, you may need to create your own mapping.
for bert_key in bert_keys:
    # pop remove the first weights from the Ordered dict ...
    _, weight = model_weights.popitem(last=False)
    # ... and we re-insert them, in order, with a new key
    model_weights[bert_key] = weight

# we re-export the weights
torch.save(model_weights, "roberta-in-bert/pytorch_model.bin")
del model_weights

We override the architecture name to make `transformers` believe it is `Bert`...

In [15]:
# =====> change architecture to bert base <======
import json

with open("roberta-in-bert/config.json") as f:
    content = json.load(f)
    content["architectures"] = ["bert"]

with open("roberta-in-bert/config.json", mode="w") as f:
    json.dump(content, f)

## Model training


When you create a classification model from a pretrained one, the last layer are randomly initialized.
We don't want to take these totally random values to compute the calibration of tensors.
Moreover, our trainset is a bit small, and it's easy to overfit.

Therefore, we train our `Roberta into Bert` model on 1/6 of the train set.
The goal is to slightly update the weights to the new architecture, not to get the best score.

> another approach is to fully train your model, perform calibration, and then retrain it on a small part of the data with a low learning rate (usually 1/10 of the original one).


In [16]:
transformers.logging.set_verbosity_error()
model_bert = BertForSequenceClassification.from_pretrained("roberta-in-bert", num_labels=num_labels)
model_bert = model_bert.cuda()

trainer = Trainer(
    model_bert,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
transformers.logging.set_verbosity_error()
trainer.train()
print(trainer.evaluate())
model_bert.save_pretrained("roberta-in-bert-trained")
del trainer
del model_bert

[INFO|trainer.py:437] 2021-12-07 16:55:07,169 >> Using amp half precision backend


KeyboardInterrupt: 

### Quantization

Below we will start the quantization process.
It follow those steps:

* perform the calibration
* perform a quantization aware training

By passing validation values to the model, we will calibrate it, meaning it will get the right range / scale to convert FP32 weights to int-8 ones.

## Calibration

### Activate histogram calibration

There are several kinds of calbrators, below we use the percentile one (99.99p) (`histogram`), basically, its purpose is to just remove the most extreme values before computing range / scale.
The other option is `max`, it's much faster but expect lower accuracy.

Second calibration option, choose between calibration done at the tensor level or per channel (more fine grained, slower).

In [ ]:
# you can also use "max" instead of "historgram"
input_desc = QuantDescriptor(num_bits=8, calib_method="histogram")
# below we do per-channel quantization for weights, set axis to None to get a per tensor calibration
weight_desc = QuantDescriptor(num_bits=8, axis=(0,))
quant_nn.QuantLinear.set_default_quant_desc_input(input_desc)
quant_nn.QuantLinear.set_default_quant_desc_weight(weight_desc)

### Perform calibration

During this step we will enable the calibration nodes, and pass some representative data to the model.
It will then be used to compute the scale/range.

Official recommendations from Nvidia is to calibrate over thousands of examples from the validation set.
Here we use 40*32 examples, because it's a slow process. It's enough to be close from the original accuracy, on your use case, follow Nvidia process.

In [ ]:
model_q = QDQBertForSequenceClassification.from_pretrained("roberta-in-bert-trained", num_labels=num_labels)
model_q = calibrate(model=model_q, encoded_dataset=encoded_dataset)

### Quantization Aware Training (QAT)

The query aware training is not a mandatory step, but **highly** recommended to get the best accuracy. Basically we will redo the training with the quantization enabled and a low learning rate.

In [ ]:
model_q = QDQBertForSequenceClassification.from_pretrained("roberta-in-bert-trained-quantized", num_labels=num_labels)
model_q = model_q.cuda()

args.learning_rate = 1e-6
trainer = Trainer(
    model_q,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
transformers.logging.set_verbosity_error()
print(trainer.evaluate())
trainer.train()
print(trainer.evaluate())
model_q.save_pretrained("roberta-in-bert-trained-quantized-bis")
del model_q
del trainer

### Benchmark

#### Export a `QDQ Pytorch` model on `ONNX`, we need to enable fake quantization mode from Pytorch.

In [23]:
data = encoded_dataset["train"][0:3]
input_torch = convert_tensor(data, output="torch")

model_q = QDQBertForSequenceClassification.from_pretrained(
    "roberta-in-bert-trained-quantized-bis", num_labels=num_labels
)
model_q = model_q.cuda()
from pytorch_quantization.nn import TensorQuantizer

TensorQuantizer.use_fb_fake_quant = True
convert_to_onnx(model_q, output_path="model_q.onnx", inputs_pytorch=input_torch)
TensorQuantizer.use_fb_fake_quant = False
# del model_q

#### Convert `ONNX` graph to `TensorRT` engine

In [14]:
engine = build_engine(
    runtime=runtime,
    onnx_file_path="model_q.onnx",
    logger=trt_logger,
    min_shape=(batch_size, max_seq_len),
    optimal_shape=(batch_size, max_seq_len),
    max_shape=(batch_size, max_seq_len),
    workspace_size=10000 * 1024 * 1024,
    fp16=False,
    int8=True,
)

In [15]:
# same thing from command line
# !/usr/src/tensorrt/bin/trtexec --onnx=model_q.onnx --shapes=input_ids:32x256,attention_mask:32x256 --int8 --workspace=10000 --saveEngine="test.plan"

#### Prepare input and output buffer

In [16]:
stream: Stream = pycuda.driver.Stream()
context: IExecutionContext = engine.create_execution_context()
context.set_optimization_profile_async(profile_index=profile_index, stream_handle=stream.handle)
input_binding_idxs, output_binding_idxs = get_binding_idxs(engine, profile_index)  # type: List[int], List[int]

In [17]:
data = encoded_dataset["train"][0:batch_size]
input_torch: Dict[str, torch.Tensor] = convert_tensor(data, output="torch")
input_np: Dict[str, np.ndarray] = convert_tensor(data, output="np")

#### Inference on `TensorRT`

We first check that inference is working correctly:

In [18]:
tensorrt_output = infer_tensorrt(
    context=context,
    host_inputs=input_np,
    input_binding_idxs=input_binding_idxs,
    output_binding_idxs=output_binding_idxs,
    stream=stream,
)
print(tensorrt_output)

[array([[ 5.6351620e-01,  1.4767665e+00, -2.0787194e+00],
       [ 2.3301950e+00, -1.0177574e+00, -1.3668290e+00],
       [ 1.5580183e+00, -5.9583592e-01, -1.1548299e+00],
       [ 1.9603873e+00, -2.0616996e-01, -1.8071964e+00],
       [ 2.4737215e+00, -2.8350648e-01, -1.8850105e+00],
       [ 3.6134090e+00, -1.3006018e-01, -2.7839746e+00],
       [-3.1495490e+00,  2.4353392e+00,  1.9919875e-01],
       [ 3.3117905e+00, -7.3505348e-01, -2.0925450e+00],
       [ 3.2750502e-01, -1.5198725e+00,  1.2251633e+00],
       [-3.7192254e+00, -5.1082242e-01,  3.6361742e+00],
       [ 3.1723669e+00, -6.5267378e-01, -2.1629393e+00],
       [-1.5052840e+00, -1.1153723e+00,  2.1314652e+00],
       [-2.7875674e+00,  3.3702278e+00, -9.6062738e-01],
       [-2.1700766e+00,  2.1553783e+00, -4.1763881e-01],
       [-1.2523253e-01, -9.4394463e-01,  8.0471390e-01],
       [ 2.3903012e+00, -1.0954552e+00, -1.0219078e+00],
       [ 3.7135108e+00, -6.1678243e-01, -2.5324042e+00],
       [-2.8983197e+00, -1.924

We warmup the GPU with few inferences and then start the measures:

In [19]:
for _ in range(30):
    _ = infer_tensorrt(
        context=context,
        host_inputs=input_np,
        input_binding_idxs=input_binding_idxs,
        output_binding_idxs=output_binding_idxs,
        stream=stream,
    )
time_buffer = list()
for _ in range(100):
    with track_infer_time(time_buffer):
        _ = infer_tensorrt(
            context=context,
            host_inputs=input_np,
            input_binding_idxs=input_binding_idxs,
            output_binding_idxs=output_binding_idxs,
            stream=stream,
        )

print_timings(name="TensorRT (INT-8)", timings=time_buffer)
del engine, context  # delete all tensorrt objects

[TensorRT (FP16)] mean=15.83ms, sd=1.68ms, min=14.22ms, max=25.70ms, median=15.09ms, 95p=17.72ms, 99p=19.29ms


## Method 2: use a dedicated QDQ model

In method 2, the idea is to take the source code of a specific model and add manually in the source code `QDQ` nodes. That way, quantization will work out of the box for this architecture.
We have started with `QDQRoberta` a quantization compliant `Roberta` model.

To adapt to another architecture, one need to:

* replace linear layers with their quantized version
* replace operations not supported out of the box by `TensorRT` by a similar code supporting the operation.

> it's not a complex process, but it requires some knowledge of both `ONNX` supported operations and `TensorRT` framework

The process below is a bit simpler than the method 1:

<!-- * finetune the QDQ model on the task (Quantization Aware Training) -->
* calibrate
* Quantization Aware training (QAT)


### Fine tuning the model

In [ ]:
# model_roberta: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(
#     model_checkpoint, num_labels=num_labels
# )
# model_roberta = model_roberta.cuda()

# args.learning_rate = 1e-5
# trainer = Trainer(
#     model_roberta,
#     args,
#     train_dataset=encoded_dataset["train"],
#     eval_dataset=encoded_dataset[validation_key],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )
# transformers.logging.set_verbosity_error()
# trainer.train()
# print(trainer.evaluate())
# # {'eval_loss': 0.3559744358062744, 'eval_accuracy': 0.8655119714722364, 'eval_runtime': 19.6678, 'eval_samples_per_second': 499.04, 'eval_steps_per_second': 7.83, 'epoch': 0.98}
# trainer.save_model("roberta-model")
# del model_roberta
# del trainer

### Calibration

In [51]:
input_desc = QuantDescriptor(num_bits=8, calib_method="histogram")
# below we do per-channel quantization for weights, set axis to None to get a per tensor calibration
weight_desc = QuantDescriptor(num_bits=8, axis=(0,))
quant_nn.QuantLinear.set_default_quant_desc_input(input_desc)
quant_nn.QuantLinear.set_default_quant_desc_weight(weight_desc)

# keep it on CPU
model_roberta: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels
)
model_roberta.save_pretrained("roberta-untrained-quantized")
del model_roberta
model_roberta_q: PreTrainedModel = QDQRobertaForSequenceClassification.from_pretrained("roberta-untrained-quantized")
model_roberta_q = calibrate(model=model_roberta_q, encoded_dataset=encoded_dataset)

model_roberta_q.save_pretrained("roberta-untrained-quantized")
del model_roberta_q

0it [00:00, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

### Quantization Aware Training (QAT)

In [52]:
model_roberta_q: PreTrainedModel = QDQRobertaForSequenceClassification.from_pretrained(
    "roberta-untrained-quantized", num_labels=num_labels
)
model_roberta_q = model_roberta_q.cuda()

args.learning_rate = 1e-5
trainer = Trainer(
    model_roberta_q,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
transformers.logging.set_verbosity_error()
trainer.train()
print(trainer.evaluate())
# {'eval_loss': 0.40235549211502075, 'eval_accuracy': 0.8589913397860418, 'eval_runtime': 46.1754, 'eval_samples_per_second': 212.559, 'eval_steps_per_second': 3.335, 'epoch': 1.0}
model_roberta_q.save_pretrained("roberta-in-bert-trained-quantized")
del model_roberta_q

[INFO|trainer.py:437] 2021-12-07 22:51:46,364 >> Using amp half precision backend


{'loss': 0.7546, 'learning_rate': 9.1875814863103e-06, 'epoch': 0.08}
{'eval_loss': 0.5080597400665283, 'eval_accuracy': 0.8026490066225166, 'eval_runtime': 46.668, 'eval_samples_per_second': 210.315, 'eval_steps_per_second': 3.3, 'epoch': 0.08}
{'loss': 0.5466, 'learning_rate': 8.372718383311604e-06, 'epoch': 0.16}
{'eval_loss': 0.452316552400589, 'eval_accuracy': 0.8242485990830362, 'eval_runtime': 46.9082, 'eval_samples_per_second': 209.238, 'eval_steps_per_second': 3.283, 'epoch': 0.16}


KeyboardInterrupt: 

### Benchmark

#### Export a `QDQ Pytorch` model on `ONNX`, we need to enable fake quantization mode from Pytorch.

In [20]:
model_roberta_q: PreTrainedModel = QDQRobertaForSequenceClassification.from_pretrained(
    "roberta-in-bert-trained-quantized-retrained", num_labels=num_labels
)
model_roberta_q = model_roberta_q.cuda()

data = encoded_dataset["train"][1:3]
input_torch = convert_tensor(data, output="torch")

from pytorch_quantization.nn import TensorQuantizer

TensorQuantizer.use_fb_fake_quant = True
convert_to_onnx(model_pytorch=model_roberta_q, output_path="roberta_q.onnx", inputs_pytorch=input_torch)
TensorQuantizer.use_fb_fake_quant = False

/home/geantvert/.local/share/virtualenvs/fast_transformer/lib/python3.9/site-packages/pytorch_quantization/nn/modules/tensor_quantizer.py:285: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  inputs, amax.item() / bound, 0,
/home/geantvert/.local/share/virtualenvs/fast_transformer/lib/python3.9/site-packages/pytorch_quantization/nn/modules/tensor_quantizer.py:291: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  quant_dim = list(amax.shape).index(list(amax_sequeeze.shape)[0])


#### Convert `ONNX` graph to `TensorRT` engine

In [21]:
engine = build_engine(
    runtime=runtime,
    onnx_file_path="roberta_q.onnx",
    logger=trt_logger,
    min_shape=(batch_size, max_seq_len),
    optimal_shape=(batch_size, max_seq_len),
    max_shape=(batch_size, max_seq_len),
    workspace_size=10000 * 1024 * 1024,
    fp16=False,
    int8=True,
)

In [29]:
# same conversion from the terminal
#!/usr/src/tensorrt/bin/trtexec --onnx=roberta_q.onnx --shapes=input_ids:32x256,attention_mask:32x256 --int8 --workspace=10000 --saveEngine="test.plan"

#### Prepare input and output buffer

In [22]:
stream: Stream = pycuda.driver.Stream()
context: IExecutionContext = engine.create_execution_context()
context.set_optimization_profile_async(profile_index=profile_index, stream_handle=stream.handle)
input_binding_idxs, output_binding_idxs = get_binding_idxs(engine, profile_index)  # type: List[int], List[int]

In [23]:
data = encoded_dataset["train"][0:batch_size]
input_torch: OD[str, torch.Tensor] = convert_tensor(data=data, output="torch")
input_np: OD[str, np.ndarray] = convert_tensor(data=data, output="np")

#### Inference on `TensorRT`

We first check that inference is working correctly:

In [24]:
tensorrt_output = infer_tensorrt(
    context=context,
    host_inputs=input_np,
    input_binding_idxs=input_binding_idxs,
    output_binding_idxs=output_binding_idxs,
    stream=stream,
)
print(tensorrt_output)

[array([[ 0.18246882,  1.1604499 , -1.9064299 ],
       [ 2.2959347 , -0.90284556, -1.7264905 ],
       [ 2.0187902 , -0.773502  , -1.5992272 ],
       [ 1.0055674 , -0.11009257, -1.5396124 ],
       [ 1.859256  , -0.2640643 , -1.9893125 ],
       [ 4.013083  , -1.0762665 , -2.6802182 ],
       [-3.8274715 ,  2.82228   ,  0.53441864],
       [ 3.9432845 , -1.111492  , -2.5691993 ],
       [-0.02036566, -1.372117  ,  0.7166275 ],
       [-3.4707005 , -1.0002614 ,  4.073874  ],
       [ 3.2287133 , -1.2782975 , -2.2006783 ],
       [-2.6840115 , -1.0188127 ,  3.1551964 ],
       [-3.7200396 ,  2.349886  ,  0.5967207 ],
       [-2.813374  ,  2.1468658 , -0.05996893],
       [-0.65687865, -0.6351316 ,  0.39051658],
       [ 2.4374492 , -1.0998904 , -1.6697674 ],
       [ 3.8417404 , -0.5648582 , -2.9105089 ],
       [-2.7713814 , -1.9245013 ,  4.5542636 ],
       [-3.3167183 , -1.1679829 ,  4.150669  ],
       [-2.3994412 ,  3.990843  , -1.8524642 ],
       [ 3.919186  , -0.6086215 , -3.06

We warmup the GPU with few inferences and then start the measures:

In [25]:
for _ in range(30):
    _ = infer_tensorrt(
        context=context,
        host_inputs=input_np,
        input_binding_idxs=input_binding_idxs,
        output_binding_idxs=output_binding_idxs,
        stream=stream,
    )
time_buffer = list()
for _ in range(100):
    with track_infer_time(time_buffer):
        _ = infer_tensorrt(
            context=context,
            host_inputs=input_np,
            input_binding_idxs=input_binding_idxs,
            output_binding_idxs=output_binding_idxs,
            stream=stream,
        )

print_timings(name="TensorRT (INT-8)", timings=time_buffer)
del engine, context

[TensorRT (FP16)] mean=14.94ms, sd=0.61ms, min=14.64ms, max=17.95ms, median=14.77ms, 95p=16.19ms, 99p=17.74ms


### TensorRT baseline

Below we export a randomly initialized `Roberta` model, the purpose is to only check the performance on mixed precision (FP16, no quantization).

In [30]:
baseline_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
baseline_model = baseline_model.cuda()
convert_to_onnx(baseline_model, output_path="baseline.onnx", inputs_pytorch=input_torch)
del baseline_model

In [35]:
engine = build_engine(
    runtime=runtime,
    onnx_file_path="baseline.onnx",
    logger=trt_logger,
    min_shape=(batch_size, max_seq_len),
    optimal_shape=(batch_size, max_seq_len),
    max_shape=(batch_size, max_seq_len),
    workspace_size=10000 * 1024 * 1024,
    fp16=True,
    int8=False,
)
stream: Stream = pycuda.driver.Stream()
context: IExecutionContext = engine.create_execution_context()
context.set_optimization_profile_async(profile_index=profile_index, stream_handle=stream.handle)
input_binding_idxs, output_binding_idxs = get_binding_idxs(engine, profile_index)  # type: List[int], List[int]
for _ in range(30):
    _ = infer_tensorrt(
        context=context,
        host_inputs=input_np,
        input_binding_idxs=input_binding_idxs,
        output_binding_idxs=output_binding_idxs,
        stream=stream,
    )
time_buffer = list()
for _ in range(100):
    with track_infer_time(time_buffer):
        _ = infer_tensorrt(
            context=context,
            host_inputs=input_np,
            input_binding_idxs=input_binding_idxs,
            output_binding_idxs=output_binding_idxs,
            stream=stream,
        )

print_timings(name="TensorRT (FP16)", timings=time_buffer)
del engine, context

[TensorRT (FP16)] mean=30.17ms, sd=0.46ms, min=29.53ms, max=32.12ms, median=29.97ms, 95p=31.24ms, 99p=31.58ms


## Pytorch baseline

To finish, we will measure vanilla Pytorch inference on both FP32 and FP16 precision, it will be our baseline:

In [40]:
baseline_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
baseline_model = baseline_model.cuda()

data = encoded_dataset["train"][0:batch_size]
input_torch: OD[str, torch.Tensor] = convert_tensor(data=data, output="torch")

for _ in range(30):
    _ = baseline_model(**input_torch)
    torch.cuda.synchronize()
time_buffer = list()
for _ in range(100):
    with track_infer_time(time_buffer):
        _ = baseline_model(**input_torch)
        torch.cuda.synchronize()
print_timings(name="Pytorch (FP32)", timings=time_buffer)

[Pytorch (FP32)] mean=76.31ms, sd=1.04ms, min=75.32ms, max=82.39ms, median=76.02ms, 95p=78.31ms, 99p=79.18ms


In [43]:
from torch.cuda.amp import autocast

with autocast():
    for _ in range(30):
        _ = baseline_model(**input_torch)
        torch.cuda.synchronize()
    time_buffer = []
    for _ in range(100):
        with track_infer_time(time_buffer):
            _ = baseline_model(**input_torch)
            torch.cuda.synchronize()
print_timings(name="Pytorch (FP16)", timings=time_buffer)

[Pytorch (FP16)] mean=56.04ms, sd=0.56ms, min=55.44ms, max=57.93ms, median=55.88ms, 95p=57.36ms, 99p=57.62ms
